In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import subprocess
 
package_name = "Faker"
 
try:
    subprocess.check_call(["pip", "install", package_name])
    print(f"Successfully installed {package_name}")
except subprocess.CalledProcessError:
    print(f"Failed to install {package_name}")


StatementMeta(, a5868af3-eff9-416e-92d2-8a984f7bcae7, 3, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.9 MB/s eta 0:00:00


In [2]:
from faker import Faker
import random
from pyspark.sql.functions import current_timestamp
import time
import json
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, BooleanType, IntegerType

fake = Faker()

store_data = [
    {"STORE_LOCATION_ID": 1, "STORE_CITY": "New York", "STORE_STATE": "New York"},
    {"STORE_LOCATION_ID": 2, "STORE_CITY": "Los Angeles", "STORE_STATE": "California"},
    {"STORE_LOCATION_ID": 3, "STORE_CITY": "Chicago", "STORE_STATE": "Illinois"},
    {"STORE_LOCATION_ID": 4, "STORE_CITY": "Houston", "STORE_STATE": "Texas"},
    {"STORE_LOCATION_ID": 5, "STORE_CITY": "San Antonio", "STORE_STATE": "Texas"},
    {"STORE_LOCATION_ID": 6, "STORE_CITY": "San Diego", "STORE_STATE": "California"},
]

warehouse_data = [
    {"WAREHOUSE_ID": 1, "WAREHOUSE_NAME": "McLane", "IS_REFRIGERATED": 1, "STORE_LOCATION_ID": 1},
    {"WAREHOUSE_ID": 2, "WAREHOUSE_NAME": "Shamrock Foods", "IS_REFRIGERATED": 0, "STORE_LOCATION_ID": 2},
    {"WAREHOUSE_ID": 3, "WAREHOUSE_NAME": "Sysco Corporation", "IS_REFRIGERATED": 1, "STORE_LOCATION_ID": 5},
    {"WAREHOUSE_ID": 4, "WAREHOUSE_NAME": "Performance Food Group", "IS_REFRIGERATED": 0, "STORE_LOCATION_ID": 4},
    {"WAREHOUSE_ID": 5, "WAREHOUSE_NAME": "Aclarity", "IS_REFRIGERATED": 0, "STORE_LOCATION_ID": 2},
    {"WAREHOUSE_ID": 6, "WAREHOUSE_NAME": "Morgan Self Storage", "IS_REFRIGERATED": 1, "STORE_LOCATION_ID": 5},
    {"WAREHOUSE_ID": 7, "WAREHOUSE_NAME": "Highwood Cold Storage", "IS_REFRIGERATED": 1, "STORE_LOCATION_ID": 6},
    {"WAREHOUSE_ID": 8, "WAREHOUSE_NAME": "Lineage Logistics", "IS_REFRIGERATED": 0, "STORE_LOCATION_ID": 3},
]


def simulate_vehicle_details():
    return {
        "vehicle_id": fake.unique.random_number(digits=5),
              
    }


def simulate_order_data():
    return {
        "order_value": random.randint(100, 1000),
        "order_contains_DairyProduct": fake.boolean(),
        "order_contains_BakeryProducts": fake.boolean(),
        "order_contains_Fruits": fake.boolean(),
        "order_contains_MeatandPoultry": fake.boolean(),
        "order_contains_Beverage": fake.boolean(),
        "order_contains_Grains": fake.boolean(),
        "order_contains_Snacks": fake.boolean(),
        "order_contains_Spices": fake.boolean()
    }   


def simulate_pickup_drop_times(vehicle_id,order_details, status, warehouse):
    current_time = int(time.time()) 
    pickup_time = current_time
    time_taken = random.randint(120, 300) * 60  
    drop_time = pickup_time + time_taken if status == "delivered" else None
    store = next((store for store in store_data if store["STORE_LOCATION_ID"] == warehouse["STORE_LOCATION_ID"]), None)
    latitude = fake.latitude() 
    longitude = fake.longitude()
    #print("This is lat",latitude)
    return {
        "timestamp": current_time,
        "vehicle_id": vehicle_id,
        "average_speed_km_hr": random.uniform(35, 95),
        "latitude": str(latitude),         
        "longitude": str(longitude),
        "warehouse_id": warehouse["WAREHOUSE_ID"],
        #"warehouse_name": warehouse["WAREHOUSE_NAME"],
        "store_location_id": store["STORE_LOCATION_ID"],
        "pickup_time": datetime.utcfromtimestamp(pickup_time).strftime('%Y-%m-%dT%H:%M:%S') if status == "picked up" else None,
        "drop_time": datetime.utcfromtimestamp(drop_time).strftime('%Y-%m-%dT%H:%M:%S') if status == "delivered" else None,
        "time_taken_hrs": time.strftime('%H:%M', time.gmtime(time_taken)) if status == "delivered" else None,
        "status": status,
        "order_value": order_details["order_value"],
        "order_contains_DairyProduct": order_details["order_contains_DairyProduct"],
        "order_contains_BakeryProducts": order_details["order_contains_BakeryProducts"],
        "order_contains_Fruits": order_details["order_contains_Fruits"],
        "order_contains_MeatandPoultry": order_details["order_contains_MeatandPoultry"],
        "order_contains_Beverage": order_details["order_contains_Beverage"],
        "order_contains_Grains": order_details["order_contains_Grains"],
        "order_contains_Snacks": order_details["order_contains_Snacks"],
        "order_contains_Spices": order_details["order_contains_Spices"]
    }

data_list = []

for id in range(1001 , 1011):
    vehicle_details = simulate_vehicle_details()
    vehicle_id =id
    order_details = simulate_order_data()
    warehouse = random.choice(warehouse_data)
    for status in ["picked up", "delivered"]:
        data_list.append(json.dumps(simulate_pickup_drop_times(vehicle_id,order_details, status, warehouse), indent=4))

schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("vehicle_id", LongType(), True),
    # StructField("vehicle_plate", StringType(), True),
    # StructField("driver_name", StringType(), True),
    StructField("average_speed_km_hr", DoubleType(), True),
    StructField("latitude", StringType(), True),
    StructField("longitude", StringType(), True),
    StructField("warehouse_id", IntegerType(), True),
    #StructField("warehouse_name", StringType(), True),
    StructField("store_location_id", IntegerType(), True),
    StructField("pickup_time", StringType(), True),
    StructField("drop_time", StringType(), True),
    StructField("time_taken_hrs", StringType(), True),
    StructField("status", StringType(), True),
    StructField("order_value", LongType(), True),
    StructField("order_contains_DairyProduct", BooleanType(), True),
    StructField("order_contains_BakeryProducts", BooleanType(), True),
    StructField("order_contains_Fruits", BooleanType(), True),
    StructField("order_contains_MeatandPoultry", BooleanType(), True),
    StructField("order_contains_Beverage", BooleanType(), True),
    StructField("order_contains_Grains", BooleanType(), True),
    StructField("order_contains_Snacks", BooleanType(), True),
    StructField("order_contains_Spices", BooleanType(), True),
])


df = spark.read.schema(schema).json(spark.sparkContext.parallelize(data_list)).orderBy(["status","time_taken_hrs"], ascending=[False, True])


display(df)



StatementMeta(, a5868af3-eff9-416e-92d2-8a984f7bcae7, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2537bb04-2cb1-4daf-8e66-6ae62b034a14)

In [3]:
df.write.mode("append").saveAsTable("real_time_data")

StatementMeta(, a5868af3-eff9-416e-92d2-8a984f7bcae7, 5, Finished, Available)

In [4]:
from pyspark.sql.types import StructType, StructField, StringType
import time

watermark_schema = StructType([
    StructField("TableName", StringType(), True),
    StructField("LastUpdated", StringType(), True)
])

from datetime import datetime
timestamp = str(int(time.time()))



values = [('real_time_data', timestamp)]
watermark_data = spark.createDataFrame(values, schema=watermark_schema)

watermark_data.show()

StatementMeta(, a5868af3-eff9-416e-92d2-8a984f7bcae7, 6, Finished, Available)

+--------------+-----------+
|     TableName|LastUpdated|
+--------------+-----------+
|real_time_data| 1704458462|
+--------------+-----------+



In [5]:
#watermark_data.write.mode("append").saveAsTable("bronze_watermark_practice")

StatementMeta(, a5868af3-eff9-416e-92d2-8a984f7bcae7, 7, Finished, Available)